# 07 - Document Loaders 文档加载

## 用途
学习使用 LangChain Document Loaders 从数据源加载文档，RAG 系统的起点

## 学习目标
- 掌握基础 Loader 使用方法
- 理解文档加载机制和结构
- 能处理本地中文文档
- 熟悉 Document 对象结构

## 代码块独立性说明
**注意**：每个代码块都是独立的，包含完整的导入和初始化，确保可以单独运行。

## 1. TextLoader 基础使用

In [ ]:
# TextLoader 基础使用 - 独立代码块
from langchain_community.document_loaders import TextLoader
import os

# 文档路径（使用相对路径）
file_path = "../sample_chinese_text.txt"

print("🔍 TextLoader 基础测试:")
print("=" * 40)

# 检查文件是否存在
if os.path.exists(file_path):
    print(f"✅ 文件存在: {file_path}")
    
    # 创建 TextLoader
    loader = TextLoader(file_path, encoding='utf-8')
    
    print(f"\n📝 Loader 创建成功")
    print(f"   文件路径: {file_path}")
    print(f"   编码格式: utf-8")
    
    # 加载文档
    documents = loader.load()
    
    print(f"\n📚 文档加载结果:")
    print(f"   文档数量: {len(documents)}")
    
    # 分析第一个文档
    if documents:
        doc = documents[0]
        print(f"   文档类型: {type(doc)}")
        print(f"   内容长度: {len(doc.page_content)} 字符")
        print(f"   元数据: {doc.metadata}")
        print(f"\n📖 文档内容预览:")
        print(f"   {doc.page_content[:100]}...")
    
    # 验证点：能正常返回中文文档内容和元数据
    assert len(documents) > 0, "文档加载失败"
    assert isinstance(documents[0].page_content, str), "文档内容不是字符串"
    assert 'source' in documents[0].metadata, "缺少源文件元数据"
    print(f"\n✅ 验证通过：TextLoader 正常工作")
    
else:
    print(f"❌ 文件不存在: {file_path}")
    print("   请确保 sample_chinese_text.txt 文件在项目根目录")

## 2. Document 对象结构分析

In [ ]:
# Document 对象结构分析 - 独立代码块
from langchain_community.document_loaders import TextLoader
import os

# 文档路径
file_path = "../sample_chinese_text.txt"

print("🔍 Document 对象结构分析:")
print("=" * 40)

if os.path.exists(file_path):
    # 加载文档
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()
    
    if documents:
        doc = documents[0]
        
        print(f"📋 Document 对象详细信息:")
        print(f"   类型: {type(doc)}")
        print(f"   模块: {type(doc).__module__}")
        
        # 分析 page_content
        content = doc.page_content
        print(f"\n📝 page_content 分析:")
        print(f"   类型: {type(content)}")
        print(f"   长度: {len(content)} 字符")
        print(f"   行数: {len(content.splitlines())} 行")
        print(f"   包含中文: {'是' if any('\u4e00' <= char <= '\u9fff' for char in content) else '否'}")
        
        # 分析 metadata
        metadata = doc.metadata
        print(f"\n🏷️  metadata 分析:")
        print(f"   类型: {type(metadata)}")
        print(f"   键值对: {list(metadata.items())}")
        
        # 检查可用的属性和方法
        print(f"\n🔧 可用属性:")
        for attr in dir(doc):
            if not attr.startswith('_') and not callable(getattr(doc, attr)):
                value = getattr(doc, attr)
                print(f"   {attr}: {type(value)} = {str(value)[:50]}...")
        
        # 验证点：Document[] 结构正确
        assert hasattr(doc, 'page_content'), "缺少 page_content 属性"
        assert hasattr(doc, 'metadata'), "缺少 metadata 属性"
        assert isinstance(doc.metadata, dict), "metadata 不是字典类型"
        print(f"\n✅ 验证通过：Document 对象结构正确")
    
else:
    print(f"❌ 文件不存在: {file_path}")

## 3. 多文档加载测试

In [ ]:
# 多文档加载测试 - 独立代码块
from langchain_community.document_loaders import TextLoader
import os
import tempfile

print("📚 多文档加载测试:")
print("=" * 40)

# 创建临时测试文件
test_files = []
test_content = [
    "这是第一个测试文档。\n包含中文内容和基础信息。",
    "这是第二个测试文档。\n包含不同的中文内容和详细信息。",
    "这是第三个测试文档。\n包含补充的中文内容和总结信息。"
]

try:
    # 创建临时文件
    temp_dir = tempfile.mkdtemp()
    for i, content in enumerate(test_content, 1):
        file_path = os.path.join(temp_dir, f"test_doc_{i}.txt")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        test_files.append(file_path)
    
    print(f"✅ 创建 {len(test_files)} 个测试文件")
    
    # 测试单个文件加载
    print(f"\n🔍 单文件加载测试:")
    for i, file_path in enumerate(test_files, 1):
        loader = TextLoader(file_path, encoding='utf-8')
        docs = loader.load()
        print(f"   文件 {i}: {len(docs)} 个文档, {len(docs[0].page_content)} 字符")
    
    # 测试批量加载
    print(f"\n📦 批量加载测试:")
    all_documents = []
    for file_path in test_files:
        loader = TextLoader(file_path, encoding='utf-8')
        all_documents.extend(loader.load())
    
    print(f"   总文档数: {len(all_documents)}")
    print(f"   总字符数: {sum(len(doc.page_content) for doc in all_documents)}")
    
    # 分析文档来源
    print(f"\n📊 文档来源分析:")
    sources = [doc.metadata.get('source', 'unknown') for doc in all_documents]
    for i, source in enumerate(sources, 1):
        filename = os.path.basename(source)
        print(f"   文档 {i}: {filename}")
    
    # 验证点：多文档加载正确
    assert len(all_documents) == len(test_files), "文档数量不匹配"
    assert all(len(doc.page_content) > 0 for doc in all_documents), "存在空文档"
    print(f"\n✅ 验证通过：多文档加载正常工作")
    
finally:
    # 清理临时文件
    import shutil
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print(f"\n🧹 清理临时文件: {temp_dir}")

## 4. 错误处理和编码测试

In [ ]:
# 错误处理和编码测试 - 独立代码块
from langchain_community.document_loaders import TextLoader
import os
import tempfile

print("🔍 错误处理和编码测试:")
print("=" * 40)

# 测试场景
test_scenarios = [
    {
        "name": "文件不存在",
        "file_path": "nonexistent_file.txt",
        "expected_error": True
    },
    {
        "name": "空文件",
        "content": "",
        "expected_error": False
    },
    {
        "name": "纯中文内容",
        "content": "这是纯中文测试内容，包含各种汉字和标点符号。",
        "expected_error": False
    },
    {
        "name": "中英文混合",
        "content": "This is English mixed with 中文内容 and 123 numbers.",
        "expected_error": False
    }
]

results = {}
temp_dir = tempfile.mkdtemp()

try:
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n🧪 测试 {i}: {scenario['name']}")
        
        try:
            # 准备文件
            if 'content' in scenario:
                file_path = os.path.join(temp_dir, f"test_{i}.txt")
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(scenario['content'])
            else:
                file_path = scenario['file_path']
            
            # 尝试加载
            loader = TextLoader(file_path, encoding='utf-8')
            documents = loader.load()
            
            # 分析结果
            if documents:
                doc = documents[0]
                print(f"   成功: {len(doc.page_content)} 字符")
                print(f"   内容预览: {doc.page_content[:50]}...")
                
                # 检查编码
                has_chinese = any('\u4e00' <= char <= '\u9fff' for char in doc.page_content)
                has_english = any('a' <= char.lower() <= 'z' for char in doc.page_content)
                print(f"   包含中文: {'是' if has_chinese else '否'}")
                print(f"   包含英文: {'是' if has_english else '否'}")
                
                results[scenario['name']] = {
                    "success": True,
                    "length": len(doc.page_content)
                }
            else:
                print(f"   成功: 但无文档内容")
                results[scenario['name']] = {"success": True, "length": 0}
            
        except Exception as e:
            if scenario['expected_error']:
                print(f"   预期错误: {type(e).__name__}")
                results[scenario['name']] = {"success": True, "expected_error": True}
            else:
                print(f"   意外错误: {e}")
                results[scenario['name']] = {"success": False, "error": str(e)}
    
    # 错误处理总结
    print(f"\n📊 错误处理测试总结:")
    successful_tests = sum(1 for result in results.values() if result["success"])
    print(f"✅ 成功测试: {successful_tests}/{len(test_scenarios)}")
    
    print("\n💡 错误处理建议:")
    print("   - 使用 try-except 捕获文件不存在错误")
    print("   - 指定正确的编码格式（utf-8 for 中文）")
    print("   - 检查文件路径和权限")
    print("   - 验证文档内容不为空")
    print("✅ 验证通过：错误处理机制有效")
    
finally:
    # 清理临时文件
    import shutil
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)

## 5. 学习总结与最佳实践

In [ ]:
# 学习总结与最佳实践 - 独立代码块
from langchain_community.document_loaders import TextLoader
import os

print("📋 Document Loaders 学习总结:")
print("=" * 40)

# 验证点检查
verification_points = [
    "✅ TextLoader 基础使用：加载本地 txt 文件",
    "✅ Document 对象结构：page_content 和 metadata",
    "✅ 多文档加载：批量处理多个文件",
    "✅ 错误处理：文件不存在和编码问题",
]

for point in verification_points:
    print(point)

print(f"\n🎯 核心技能掌握情况: {len(verification_points)}/4 项")

print("\n💡 Document Loaders 最佳实践:")
print("1. 文件路径：使用相对路径或绝对路径要明确")
print("2. 编码格式：中文文档使用 utf-8 编码")
print("3. 错误处理：检查文件存在性和读取权限")
print("4. 批量处理：使用循环处理多个文档")
print("5. 内容验证：检查文档内容长度和格式")

print("\n🚀 下一步学习建议:")
print("1. 深入学习 Text Splitters 文档分割")
print("2. 掌握其他 Loader 类型（PDF、CSV、JSON）")
print("3. 学习 DirectoryLoader 目录批量加载")
print("4. 探索自定义 Loader 开发")
print("5. 实践大文档处理和优化策略")

# 最终验证：确保 TextLoader 基础功能可用
try:
    file_path = "../sample_chinese_text.txt"
    
    if os.path.exists(file_path):
        loader = TextLoader(file_path, encoding='utf-8')
        documents = loader.load()
        
        print(f"\n🎉 最终验证成功:")
        print(f"   加载文件: {os.path.basename(file_path)}")
        print(f"   文档数量: {len(documents)}")
        print(f"   内容长度: {len(documents[0].page_content)} 字符")
        print("\n✅ Document Loaders 学习完成！")
    else:
        print(f"\n⚠️  最终验证跳过: 测试文件不存在")
        
except Exception as e:
    print(f"\n❌ 最终验证失败: {e}")